# PDI

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from os import listdir


In [ ]:
def logGabor(lamda, theta, angl, sigma, thetaSigma, M, N):
    #cria o meshgrid radial
    (x, y) = np.meshgrid(np.arange(-N//2 + 1, N//2 + 1)/N, np.arange(-M//2 + 1, M//2 + 1)/M)
    radius = np.sqrt(x**2 + y**2)
    radius[N//2, M//2] = 1  # Evita a divisão por zero, mas sem definir um valor fixo

    #radial component
    fo = 1/lamda # desired filter wavelength
    logGabor = np.exp(-0.5 * (np.log(radius/fo)**2)/np.log(sigma)**2) # The bandwidth of the filter is controlled by the parameter sigma
    logGabor [N//2, M//2] = 0

    #frequency component
    theta = np.arctan2(-y, x)
    sinTheta = np.sin(theta)
    cosTheta = np.cos(theta)

    ds = sinTheta * np.cos(angl) - cosTheta * np.sin(angl)
    dc = cosTheta * np.cos(angl) + sinTheta * np.sin(angl)
    dtheta = abs(np.arctan2(ds, dc))
    
    spread = np.exp(-0.5 * (dtheta)**2/thetaSigma**2)

    #Combinign
    gab_filter = logGabor * spread


    return gab_filter 

In [ ]:
def filter(imgFreq, gb, P, Q, M, N):
    #G = H*F
    Fimg_filtered = gb*imgFreq

    #fft inversa
    img_back = np.fft.ifft2(Fimg_filtered)

    # Multiplicao por (-1)^(x+y)
    mult_img_back = np.zeros([P, Q])
    for x in range(P):
        for y in range(Q):
            mult_factor = (-1)**(x+y)
            mult_img_back[x, y] = img_back[x, y].real * mult_factor

    final_img = mult_img_back[0:M, 0:N]

    return final_img

In [ ]:
img_path = 'C:/Users/mathe/Desktop/Pastas/UFSCar/Perfil 5/PDI/Swedish Leaf Dataset/leafs 1 to 3'

df = pd.DataFrame()

In [ ]:
i = 1
classes = []

#para cada imagem:
for img_filename in listdir(img_path):
    print(f'{i}: Processing imag: {img_path}/{img_filename}')
    # Lê imagem e automaticamente converte para 1 canal de cinza
    img = cv2.imread(img_path + '/' + img_filename, cv2.IMREAD_GRAYSCALE)

    img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)

    img2 = img.reshape(-1)
    df_image = pd.DataFrame()
    df_image['Original Image'] = img2

    #Padding 
    M, N = img.shape
    img_padded = np.pad(img, ((0, M), (0, N)), 
                        mode='constant', constant_values=0)
    P, Q = img_padded.shape


    # Multiplicao por (-1)^(x+y)
    mult_img = np.zeros([P, Q])

    #for x in range(P):
    #    for y in range(Q):
    #        mult_factor = (-1)**(x+y)
    #        mult_img[x, y] = img_padded[x, y] * mult_factor
    mult_img = img_padded * np.fromfunction(lambda x, y: (-1)**(x+y), img_padded.shape)

    # Converter imagem do domínio do espaço para o domínio da frequência
    f = np.fft.fft2(mult_img)

    # =====================================
    count = 1
    #Cria filtros de gabor
    classes.append(img_filename.split('nr')[0])
    for theta in range(3):
        theta = theta * 60 * (np.pi/180)
        for sigma in (1.8, 1.8):
            for lamba  in (25, 50):
                for gamma in (0.5, 1):
                    label = 'Gabor-' + str(count)

                    psi = 0
                    gb = logGabor(lamba, theta, psi, sigma, gamma, P, Q)

                    fimg = filter(f, gb, P, Q, M, N)
                    filtered_img = fimg.reshape(-1)
                    df_image[label] = filtered_img
                    count += 1

    count = 0
    flattened = df_image.values.reshape(1, -1)
    df = pd.concat([df, pd.DataFrame(flattened)], ignore_index=True)
    i += 1
df.insert(0, 'classe', classes)
df